In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

In [ ]:

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

In [ ]:
from unidecode import unidecode
import re
import warnings
warnings.filterwarnings('ignore')


def llm_bot(patient_summery):
  #below part will load the model and asper assigned rules we assign model that response in this format

  messages = [
      {"role": "system", "content": "You are a highly skilled AI assistant specialized in medical data extraction."},
      {"role": "user", "content": '''i. I will provide you with a patient summary. Please extract the following information and return it in a clean format: name, gender, age, weight, height, BMI, and chief medical complaint.
                                    ii. For the BMI, you need to calculate the BMI using formula and provide only calculated BMI number in response without any other information.
                                    iii. Do not add any extra information except asked like used formulas.'''},

      {"role": "assistant", "content": '''
        name - extracted_name,
        gender- extracted_gender,
        age - extracted_age,
        weight - extracted_weight,
        height - extracted_height,
        BMI - caculated_BMI,
        chief_medical_complaint - extracted_chief_medical_complaint
        '''},

      {"role": "user", "content": f"Patient Summary: {patient_summery}"}
      ]

  pipe = pipeline(
      "text-generation",
      model=model,
      tokenizer=tokenizer,
  )

  # Define generation parameters for the model's output
  generation_args = {
      "max_new_tokens": 500,
      "return_full_text": False,
      "temperature": 0.0,
      "do_sample": False,
  }

  output = pipe(messages, **generation_args)
  result = output[0]['generated_text']

  #As we get the response from model we are postprocessing it which includes clearning, and converting into dictionary
  if result:

    result = unidecode(result)

    result = re.sub(r'\n', '', result)
    result = re.sub(r'  ', '', result)

    required_info = {}

    #Below we are using regex pattern to fetch required details
    #-- pattern for name
    pattern = r'name - ([A-Za-z\s\.]+),'
    match = re.search(pattern, result)
    if match:
      required_info['name'] = match.group(1)


    #-- pattern for gender
    pattern = r'gender - ([A-Za-z\s\.]+),'
    match = re.search(pattern, result)
    if match:
      required_info['gender'] = match.group(1)


    #-- pattern for age
    pattern = r'age - ([\d{1,3}]+),'
    match = re.search(pattern, result)
    if match:
      required_info['age'] = match.group(1)


    #-- pattern for weight
    pattern = r'weight - (\d+ lbs),'
    match = re.search(pattern, result)
    if match:
      required_info['weight'] = match.group(1)

    #-- pattern for height
    pattern = r'height - (\d+ inches)'
    match = re.search(pattern, result)
    if match:
      required_info['height'] = match.group(1)

    #-- pattern for BMI
    pattern = r'BMI - ([\d{1,3.}]+)'
    match = re.search(pattern, result)
    if match:
      required_info['BMI'] = match.group(1)


    #-- pattern for chief medical complaint
    pattern = r'chief_medical_complaint - (.*).'
    match = re.search(pattern, result)
    if match:
      required_info['chief_medical_complaint'] = match.group(1)

  return required_info

In [ ]:
#Provide single string and get output

patient_summery = '''John R. Whitaker, a 52-year-old male, stands 5'10" (70 inches) tall and weighs 198 lbs. Mr. Whitaker has a history of hypertension and type 2 diabetes, both diagnosed in his mid-40s, and recently began experiencing worsening peripheral neuropathy in his lower extremities. He also reports chronic lower back pain, which he attributes to years of heavy lifting in his previous occupation as a construction worker. Over the past six months, John has developed shortness of breath during mild exertion, prompting concerns about potential early-stage congestive heart failure. Additionally, he struggles with obesity-related sleep apnea, contributing to fatigue and cognitive fog throughout the day. Despite his conditions, Mr. Whitaker maintains a generally positive outlook but admits to inconsistent medication adherence and difficulty following a healthy diet'''
required_info = llm_bot(patient_summery)
print("required_info =", required_info)

In [ ]:
print(type(required_info))

In [ ]:
#Provide multiple patient summery as free text, and get output for each text.

patient_summeries = [
    '''John R. Whitaker, a 52-year-old male, stands 5'10" (70 inches) tall and weighs 198 lbs. Mr. Whitaker has a history of hypertension and type 2 diabetes, both diagnosed in his mid-40s, and recently began experiencing worsening peripheral neuropathy in his lower extremities. He also reports chronic lower back pain, which he attributes to years of heavy lifting in his previous occupation as a construction worker. Over the past six months, John has developed shortness of breath during mild exertion, prompting concerns about potential early-stage congestive heart failure. Additionally, he struggles with obesity-related sleep apnea, contributing to fatigue and cognitive fog throughout the day. Despite his conditions, Mr. Whitaker maintains a generally positive outlook but admits to inconsistent medication adherence and difficulty following a healthy diet''',
    '''For the past six months, Emily J. Rivera has been dealing with persistent chest tightness and shortness of breath, especially during moderate activity. She works as a schoolteacher and describes her symptoms as worsening under stress, which she initially dismissed as anxiety. A thorough evaluation revealed mild asthma, along with borderline high cholesterol levels. Emily is 41 years old, 5'5" (65 inches) tall, and weighs 172 lbs. She also complains of intermittent joint stiffness in her hands, particularly in the mornings, which her physician suspects could be early osteoarthritis. Emily’s sedentary lifestyle and inconsistent exercise routine have contributed to her struggles with maintaining a healthy weight, though she remains committed to improving her overall health with proper guidance and treatment.''',
    '''Karen L. Thompson, a 38-year-old female, is 5'4" (64 inches) tall and weighs 162 lbs. She has a history of irritable bowel syndrome (IBS) and recurrent migraines, both of which have intensified over the past year. Karen also experiences chronic fatigue and joint pain, leading her physician to investigate possible early-stage rheumatoid arthritis. She reports frequent episodes of dizziness and occasional heart palpitations, which have been attributed to mild anemia and elevated stress levels. Karen’s symptoms are exacerbated by her demanding job as a paralegal, where long hours and poor posture have contributed to persistent neck and shoulder tension. Recently, she has begun experiencing intermittent insomnia, further impacting her energy levels and overall well-being.'''
]

for patient_summery in patient_summeries:
  print('--------------------')
  required_info = llm_bot(patient_summery)
  print("required_info =", required_info)

In [ ]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True